# Installations
Installation of the necessary libraries

In [1]:
%%capture
# comment out the above line to display output

!apt install -y swig

%pip install --upgrade pip
%pip install poetry
%pip install 'git+https://gitlab.ethz.ch/bsaverio/coco-project.git@main'

# for rendering video
%pip install pyvirtualdisplay imageio-ffmpeg "moviepy>=1.0"
!apt-get install -y xvfb python-opengl ffmpeg

Import of the necessary libraries and creatrion of useful functions

In [2]:
# set up `show_video` function for Google Colab
import base64
import glob
import io
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium as gym
import numpy as np
import coco_rocket_lander
from coco_rocket_lander.env import SystemModel
import cvxpy as cp

# create a display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# play back the video
def show_video(index: int = None):
  mp4list = sorted(glob.glob('video/*.mp4'))  # we modify the provided function to sort, for convenience
  if len(mp4list) > 0:
    vid_index = index if index is not None else 0
    mp4 = mp4list[vid_index]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")

# small convenience function to show which index corresponds to which video
def list_videos():
  for idx, video in enumerate(sorted(glob.glob("video/*.mp4"))):
    print(f"Index: {idx} = {video}")

# PID Controller

The following code initializes a basic PID-type controller

In [3]:
from coco_rocket_lander.algs import PID_Benchmark

engine_pid_params = [10, 0, 10]
engine_vector_pid_params = [0.085, 0.001, 10.55]
side_engine_pid_params = [5, 0, 6]

pid = PID_Benchmark(engine_pid_params, engine_vector_pid_params, side_engine_pid_params)

Simulation of the PID controller under the "failure mode"

In [4]:
# x_0 = np.array([0.3,0.9, 5, -5, 0.4, -0.0]) #initial state for CURRENT STATE section
x_0 = np.array([0.4,0.9, 0, -5, -0.0, -0.0]) #initial state for FAILURE MODE section
args = {
    "initial_state": tuple(x_0)
}

env = gym.make("coco_rocket_lander/RocketLander-v0", render_mode="rgb_array", args=args)
env = gym.wrappers.RecordVideo(env, 'video', episode_trigger = lambda x: True, name_prefix="Lilian_Laporte_PID_Failure_Case")

obs, info = env.reset(seed=0)  # specify a random seed for consistency

# specify the target
target_x = env.cfg.width / 2
target_y = env.cfg.height / 2
landing_position = env.get_landing_position()
target_x = landing_position[0]
target_y = landing_position[1]

# simulate
while True:
    # offset so that (0, 0) is at the target position
    pid_obs = obs
    pid_obs[0] = obs[0] - target_x
    pid_obs[1] = obs[1] - target_y

    # get action
    action = np.array(pid.pid_algorithm(pid_obs))

    next_obs, rewards, done, _, info = env.step(action)

    # check if simulation ended
    if done:
        print('reward:',rewards)
        break

    # update observation
    obs = next_obs

env.close()  # video saved at this step

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:79: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/video/Lilian_Laporte_PID_Failure_Case-episode-0.mp4.
Moviepy - Writing video /content/video/Lilian_Laporte_PID_Failure_Case-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/Lilian_Laporte_PID_Failure_Case-episode-0.mp4
reward: -100


In [11]:
list_videos()
show_video(1) #show the video

Index: 0 = video/Lilian_Laporte_Controller_Demonstration-episode-0.mp4
Index: 1 = video/Lilian_Laporte_PID_Failure_Case-episode-0.mp4


**How the failure scenario has been modeled in the simulation?** <br>
The failure scenario has been designed so that the PID controller fails to land the rocket. Slightly more challenging initial conditions are sufficient to reach the controller's limit and cause the rocket to crash. To check that this is a failure case (i.e. rocket legs make contact with the sea, or the rocket body contacts anything), the environment returns *reward=-100* and *done=True*. <br> <br>
**What parameters are available to define the failure scenario?** <br>
The parameters available are the initial conditions of the rocket (initial position, initial velocity, and initial angle and velocity angle). In our case, the initial state is $x_0 = [0.4,0.9, 0, -5, -0, -0]$. This corresponds to a vertical arrival of the rocket slightly off-center from the platform, which is plausible in real-life situations.



# Tracking MPC

Creation of the class MPC

In [7]:
class MPC:
    def __init__(self, K, n_x, n_u, A, B, Q, R, x_f, u_f, gravity_comp_fraction):
        self.K = K
        self.Q = Q
        self.R = R

        self.n_x = n_x
        self.n_u = n_u
        self.A = A
        self.B = B
        self.x_f = x_f
        self.u_f = u_f

        self.gravity_comp_fraction = gravity_comp_fraction

    def solve(self, x_0):
        x = cp.Variable((self.n_x, self.K + 1))
        u = cp.Variable((self.n_u, self.K))

        cost = 0
        constr = []
        for k in range(self.K):
            cost +=  cp.quad_form(x[:, k]-self.x_f,self.Q) + \
                     cp.quad_form(u[:, k]-[self.gravity_comp_fraction,0,0]-self.u_f,self.R)
            constr += [x[:, k + 1] == self.A @ x[:, k] + self.B @ (u[:, k]-[self.gravity_comp_fraction,0,0]),
                      x[0,k] >= 0, x[0,k] <= 33.333,
                      x[1,k] >= 0, x[1,k] <= 26.666,
                      x[4,k] >= -0.6108, x[4,k] <= 0.6108,
                       u[0,k] >= 0, u[0,k] <= 1,
                       u[1,k] >= -1, u[1,k] <= 1,
                       u[2,k] >= -1, u[2,k] <= 1]

        # sums problem objectives and concatenates constraints.
        constr += [x[:, 0] == x_0]
        problem = cp.Problem(cp.Minimize(cost), constr)
        problem.solve()

        # # Extract and return the optimal control sequence
        if problem.status == cp.OPTIMAL:
            return u[:, 0].value
        else:
            # raise Exception("Optimization problem did not converge.")
            return [0]

Simulation of the tracking MPC controller under the "failure mode"

In [8]:
# x_0 = np.array([0.3,0.9, 5, -5, 0.4, -0.0]) #initial state for CURRENT STATE section
x_0 = np.array([0.4,0.9, 0, -5, -0.0, -0.0]) #initial state for DEMONSTRATION section
args = {
    "initial_state": tuple(x_0)
}

env = gym.make("coco_rocket_lander/RocketLander-v0", render_mode="rgb_array", args=args)
env = gym.wrappers.RecordVideo(env, 'video', episode_trigger = lambda x: True, name_prefix="Lilian_Laporte_Controller_Demonstration")

# get the system model
model = SystemModel(env)
model.discretize_system_matrices(sample_time=1/60)  # set to the environment update rate
A, B = model.get_discrete_linear_system_matrices()
n_x, n_u = 6, 3 #State and Input dimension

# calculate the gravity compensation, subtract from applied actions
gravity = env.cfg.gravity
mass, inertia = env.get_mass_properties()
gravity_comp_fraction = -gravity * mass / env.cfg.main_engine_thrust

# Tuned parameters
K = 50   # prediciton horizon
Q = np.diagflat([2,2,1,1,3,2])  #State Cost
R = np.diagflat([2,1,3])  #Input Cost

#final state and input
landing_position = env.get_landing_position()
x_f = np.concatenate((landing_position[:2],[0,0,0,0]), axis=None) #x_f = [x_f y_f 0 0 0 0]
u_f = [0,0,0]

controller = MPC(K, n_x, n_u, A, B, Q, R, x_f, u_f, gravity_comp_fraction) #creation of the controller
obs, info = env.reset(seed=0)  # specify a random seed for consistency

# simulate
while True:
    action = controller.solve(obs[:6])
    if len(action)==1:
      done = True
    else:
      next_obs, rewards, done, _, info = env.step(action)

    # check if simulation ended
    if done:
        print('reward:', rewards)
        break

    # update observation
    obs = next_obs

env.close()  # video saved at this step

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:79: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Note: A and B matrices have not been initialized. Using the (default) upright equilibrium
Moviepy - Building video /content/video/Lilian_Laporte_Controller_Demonstration-episode-0.mp4.
Moviepy - Writing video /content/video/Lilian_Laporte_Controller_Demonstration-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/Lilian_Laporte_Controller_Demonstration-episode-0.mp4
reward: 100


In [10]:
list_videos()
show_video(0) #show the video

Index: 0 = video/Lilian_Laporte_Controller_Demonstration-episode-0.mp4
Index: 1 = video/Lilian_Laporte_PID_Failure_Case-episode-0.mp4


**How the proposed controller is implemented?** <br>
The controller used here is a Tracking MPC. It solves the following OPC at each time step: <br>
\begin{array}{ll} \mbox{minimize} & \sum_{k=0}^{K-1} \|x_k -x_s \|^2_Q + \| u_k -u_s\|_R^2\\
\mbox{subject to} & x_{k+1} = Ax_k + Bu_k\\%, \quad k=0, \ldots, K-1\\
&  \begin{bmatrix} 0 \\ -1 \\ -1 \end{bmatrix} \leq u_k\leq \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix}, \quad \begin{bmatrix} 0 \\ 0 \\ -\infty \\ -\infty \\ -0.6108 \\ -\infty \end{bmatrix} \leq x_k\leq \begin{bmatrix} 33.333 \\ 26.666 \\ \infty \\ \infty \\ 0.6108 \\ \infty \end{bmatrix}.
\end{array}
This MPC is implemented using a class *MPC*. 2 simple functions are used in this class: ```__init__``` that initializes all the variables needed for the optimization, and ```solve``` that resolves the optimization problem using the *cvxpy* library. If the problem is feasible, the latter returns the correct next input, but otherwise it returns a list ```[0]``` that will be interpreted as a condition to stop the simulation later in the code. <br> <br>

**What parameters need to be tuned in the controller?** <br>
3 parameters need to be tuned for the tracking MPC:
*   The prediction horizon *K*: an integer that corresponds to the number of future control intervals the MPC controller must evaluate by prediction when doing the optimization
*   The state cost matrix *Q*: a matrix of size 6x6 with the diagonal respectively corresponding to the cost of the state $[x, y, \dot{x}, \dot{y}, \theta, \dot{\theta}]$ (here we do not take into account the booleans $c_{L}$ and $c_{R}$)
*   The input cost matrix *R*: a matrix of size 3x3 with the diagonal respectively corresponding to the cost of the input $[F_{E}, F_{S}, 𝛷]$
<br> <br>

**How you tuned them and how would you recommend tuning them?** <br>
*   I've tuned K so that the controller has enough margin to anticipate the
rocket's complex maneuvers. I tried to reduce it as much as possible so that the control would be as fast as possible.
*   For the matrix Q, I put the highest cost (=3) on the angle of the rocket in order to avoid the rocket to be close to the limit ($\pm$ 35 degrees). For the position and the angular velocity, the cost is intermediate (=2) in order to force the rocket to land on the landing position without being too brutal in the rotations. Finally, the x and y velocities have the lowest cost (=1) because the rocket must be allowed a certain speed so that it can land properly.
*   For the matrix R, the highest cost (=3) is for the angle of the nozzle. It avoids the rocket to move too abruptly ang getting carried away. A smaller cost (=2) is assigned to the thrust of the main engine in order to keep a smooth acceleration of the rocket. Finally, the side gas thrusters have the lowest cost (=1) so that the rocket keeps a nice mobility and manoeuvrability.
